In [ ]:
!pip install beautifulsoup4
!pip install soccerplots

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import bs4
import requests
import re
import matplotlib.pyplot as plt
from pprint import pprint
import json

In [ ]:
# pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 11)

## Match History

### Flow
1. construct table of next matches-to-be-played
2. query home vs away match
3. check 


### Table
1. Table 1: Match Info 
  - if the match have been played, then fill the Table 2
  - columns
    - league (str)
    - season (str)
    - home (str)
    - away (str)
    - recorded (yes/no/on_watch)

2. Table of Match Results & Information
  - contains all the necessary information
  - match general infos
    - all in fbrefs
  - both teams infos prior to match, with columns:
    - all in fbrefs
  - both teams' fielded-players infos, with columns:
    - all in fbrefs
    - starter (yes/no)
    - minutes_played (int)

3. Table of Player Stats



In [ ]:
def get_html_document(url):
  # request for HTML document of given url
  response = requests.get(url)
  # response will be provided in JSON format
  return response.text

In [ ]:
## Scrap Match Result

# league_match_history_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
league_match_history_url = "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"

html_doc = get_html_document(league_match_history_url)
soup = bs4.BeautifulSoup(html_doc, 'html.parser')

In [ ]:
pl_scores_fixtures = pd.read_html(league_match_history_url)

In [ ]:
df_fixtures = pd.DataFrame(pl_scores_fixtures[0])

In [ ]:
table_1 = df_fixtures[df_fixtures["Match Report"] == "Head-to-Head"]

In [ ]:
table_2 = (
    df_fixtures[
        (df_fixtures["Notes"] != "Match Postponed") &
        (df_fixtures["Score"].isna())
    ]
    .drop(columns=["Notes"])
    .dropna(axis = 0, how = 'all')
)

In [ ]:
# ## next few matches

# clubs_recorded = []
# df_ = []

# for idx, row in df_fixtures_clean.iterrows():
#   club1 = row["Home"]
#   club2 = row["Away"]
  
#   if club1 in clubs_recorded or club2 in clubs_recorded:
#     continue

#   df_.append(row)
#   clubs_recorded.append(club1)
#   clubs_recorded.append(club2)

In [ ]:
# list to store tuples of next matches
# format: (home_team, away_team)
next_matches = []

fixtures_soup = soup.find_all('table')[0]

# find all the matches that haven't been played yet
# marked by available links to head-to-head
for link in fixtures_soup.find_all('a', attrs={'href': re.compile("^/en/stathead/")})[:]:
  home_away_hist = link.get('href').split('/')[-1]
  home_away = home_away_hist.replace('-History', '')

  # remove derby words if exists
  home_away = home_away.split("-Derby-")[-1]

  home = home_away.split('-vs-')[0].replace('-', ' ')
  away = home_away.split('-vs-')[1].replace('-', ' ')

  next_matches.append((home, away))

## Create Tables

In [ ]:
## Scrap Match Result

league_match_history_url = "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"

html_doc = get_html_document(league_match_history_url)
soup = bs4.BeautifulSoup(html_doc, 'html.parser')

### Table 1

In [ ]:
pl_fixtures = pd.read_html(league_match_history_url)

df_table_1 = pd.DataFrame(pl_fixtures[0])
df_table_1 = df_table_1.dropna(axis=0, how='all').reset_index().drop(columns=["index"])
df_table_1 = df_table_1[df_table_1["Notes"] != "Match Postponed"]

In [ ]:
df_table_1["Match Report Link"] = 'empty'

In [ ]:
TEAM_MAPPING = {
    "Arsenal": "Arsenal",
    "Aston Villa": "Aston-Villa",
    "Bournemouth": "Bournemouth",
    "Brentford": "Brentford",
    "Brighton": "Brighton-and-Hove-Albion",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal-Palace",
    "Everton": "Everton",
    "Fulham": "Fulham",
    "Leeds United": "Leeds-United",
    "Leicester City": "Leicester-City",
    "Liverpool": "Liverpool",
    "Manchester City": "Manchester-City",
    "Manchester Utd": "Manchester-United",
    "Newcastle Utd": "Newcastle-United",
    "Nott'ham Forest": "Nottingham-Forest",
    "Southampton": "Southampton",
    "Tottenham": "Tottenham-Hotspur",
    "West Ham": "West-Ham-United",
    "Wolves": "Wolverhampton-Wanderers",
}

In [ ]:
epl = (9, 'Premier-League')
la_liga = (12, 'La-Liga')
serie_a = (11, 'Serie-A')
ligue_1 = (13, 'Ligue-1')
bundesliga = (20, 'Bundesliga')
LEAGUES = [epl, la_liga, serie_a, ligue_1, bundesliga]
db = {}

In [ ]:
def scrap_league(league_id, league_name):
  league_url = f'https://fbref.com/en/comps/{league_id}/{league_name}-Stats'
  html_doc = get_html_document(league_url)
  soup = bs4.BeautifulSoup(html_doc, 'html.parser')

  lg_table_soup = soup.find_all('table')[0]

  return lg_table_soup

In [ ]:
lg_soup = scrap_league(la_liga[0], la_liga[1])

In [ ]:
team_urls = []

for link in lg_soup.find_all('a', attrs={'href': re.compile("^/en/squads/")}):
  team_url = link.get('href')
  team_urls.append(team_url)

# get unique lists of items
team_urls = list(set(team_urls))

In [ ]:
# team_urls

In [ ]:
team_kv_pairs = {}

for link in team_urls:
  # get rid of '/'
  texts = link.split('/')
  # get team id
  id = texts[-2]
  # get team name
  team_name = texts[-1][:-6]
  if id == '' or team_name == '':
    continue

  team_kv_pairs[team_name] = id

In [ ]:
{key:team_kv_pairs[key] for key in sorted(team_kv_pairs.keys())}

{'Almeria': '78ecf4bb',
 'Athletic-Club': '2b390eca',
 'Atletico-Madrid': 'db3b9613',
 'Barcelona': '206d90db',
 'Cadiz': 'ee7c297c',
 'Celta-Vigo': 'f25da7fb',
 'Elche': '6c8b07df',
 'Espanyol': 'a8661628',
 'Getafe': '7848bd64',
 'Girona': '9024a00a',
 'Mallorca': '2aa12281',
 'Osasuna': '03c57e2b',
 'Rayo-Vallecano': '98e8af82',
 'Real-Betis': 'fc536746',
 'Real-Madrid': '53a2f082',
 'Real-Sociedad': 'e31d1cd9',
 'Sevilla': 'ad2be733',
 'Valencia': 'dcc91a7b',
 'Valladolid': '17859612',
 'Villarreal': '2a8183b3'}

In [ ]:
past_matches = []
temp_soup = soup.find_all('table')[0]

for idx, row in df_table_1[df_table_1["Match Report"] == "Match Report"].iterrows():
  home = row["Home"]
  home = TEAM_MAPPING[home]
  away = row["Away"]
  away = TEAM_MAPPING[away]

  past_matches.append((home, away))

  for link in temp_soup.find_all('a', attrs={'href': re.compile("^/en/matches/")}):
    # ex: "/en/matches/073227b6/Merseyside-Derby-Everton-Liverpool-September-3-2022-Premier-League"
    last_link = link.get('href').split('/')[-1]

    # ex: "Merseyside-Derby-Everton-Liverpool-September-3-2022-Premier-League"
    if "Premier-League" not in last_link:
      continue

    last_link = last_link.split("-Derby-")[-1]
    # ex: "Everton-Liverpool-September-3-2022-Premier-League"
    
    if last_link.startswith(f"{home}-{away}"):
      df_table_1.loc[idx, "Match Report Link"] = link.get('href')
      break

In [ ]:
# df_table_1[df_table_1["Match Report Link"] != "empty"]
df_table_1

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN,/en/matches/6713c1dc/Fulham-Liverpool-August-6...
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN,/en/matches/09d8a999/Tottenham-Hotspur-Southam...
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,Match Report,NaN,/en/matches/1ac96eb4/Newcastle-United-Nottingh...
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN,/en/matches/82702941/Leeds-United-Wolverhampto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN,empty
376,38.0,Sun,2023-05-28,15:00,Leicester City,NaN,NaN,NaN,West Ham,NaN,King Power Stadium,NaN,Head-to-Head,NaN,empty
377,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN,empty
378,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN,empty


### Table 2

In [ ]:
df_table_2 = df_table_1.copy()

df_next_matches = df_table_2[df_table_2["Match Report"] != "Match Report"]

In [ ]:
## next few matches

clubs_recorded = []
df_ = []

for idx, row in df_next_matches.iterrows():
  club1 = row["Home"]
  club2 = row["Away"]
  
  if club1 in clubs_recorded or club2 in clubs_recorded:
    continue

  df_.append(row)
  clubs_recorded.append(club1)
  clubs_recorded.append(club2)

df_ = pd.DataFrame(df_)

In [ ]:
df_table_2.loc[df_.index, "Match Report Link"] = "Watch"

In [ ]:
df_table_2[df_table_2["Match Report Link"] == "Watch"]

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
198,7.0,Thu,2023-01-19,20:00,Manchester City,NaN,NaN,NaN,Tottenham,NaN,Etihad Stadium,NaN,Head-to-Head,NaN,Watch
199,21.0,Sat,2023-01-21,12:30,Liverpool,NaN,NaN,NaN,Chelsea,NaN,Anfield,NaN,Head-to-Head,NaN,Watch
200,21.0,Sat,2023-01-21,15:00,Bournemouth,NaN,NaN,NaN,Nott'ham Forest,NaN,Vitality Stadium,NaN,Head-to-Head,NaN,Watch
201,21.0,Sat,2023-01-21,15:00,Southampton,NaN,NaN,NaN,Aston Villa,NaN,St. Mary's Stadium,NaN,Head-to-Head,NaN,Watch
202,21.0,Sat,2023-01-21,15:00,Leicester City,NaN,NaN,NaN,Brighton,NaN,King Power Stadium,NaN,Head-to-Head,NaN,Watch
203,21.0,Sat,2023-01-21,15:00,West Ham,NaN,NaN,NaN,Everton,NaN,London Stadium,NaN,Head-to-Head,NaN,Watch
204,21.0,Sat,2023-01-21,17:30,Crystal Palace,NaN,NaN,NaN,Newcastle Utd,NaN,Selhurst Park,NaN,Head-to-Head,NaN,Watch
206,21.0,Sun,2023-01-22,14:00,Leeds United,NaN,NaN,NaN,Brentford,NaN,Elland Road,NaN,Head-to-Head,NaN,Watch
207,21.0,Sun,2023-01-22,16:30,Arsenal,NaN,NaN,NaN,Manchester Utd,NaN,Emirates Stadium,NaN,Head-to-Head,NaN,Watch
240,25.0,Fri,2023-02-24,20:00,Fulham,NaN,NaN,NaN,Wolves,NaN,Craven Cottage,NaN,Head-to-Head,NaN,Watch


#### Team Columns

In [ ]:
df_watches = df_table_2[df_table_2["Match Report Link"] == "Watch"]

In [ ]:
df_watches

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
198,7.0,Thu,2023-01-19,20:00,Manchester City,NaN,NaN,NaN,Tottenham,NaN,Etihad Stadium,NaN,Head-to-Head,NaN,Watch
199,21.0,Sat,2023-01-21,12:30,Liverpool,NaN,NaN,NaN,Chelsea,NaN,Anfield,NaN,Head-to-Head,NaN,Watch
200,21.0,Sat,2023-01-21,15:00,Bournemouth,NaN,NaN,NaN,Nott'ham Forest,NaN,Vitality Stadium,NaN,Head-to-Head,NaN,Watch
201,21.0,Sat,2023-01-21,15:00,Southampton,NaN,NaN,NaN,Aston Villa,NaN,St. Mary's Stadium,NaN,Head-to-Head,NaN,Watch
202,21.0,Sat,2023-01-21,15:00,Leicester City,NaN,NaN,NaN,Brighton,NaN,King Power Stadium,NaN,Head-to-Head,NaN,Watch
203,21.0,Sat,2023-01-21,15:00,West Ham,NaN,NaN,NaN,Everton,NaN,London Stadium,NaN,Head-to-Head,NaN,Watch
204,21.0,Sat,2023-01-21,17:30,Crystal Palace,NaN,NaN,NaN,Newcastle Utd,NaN,Selhurst Park,NaN,Head-to-Head,NaN,Watch
206,21.0,Sun,2023-01-22,14:00,Leeds United,NaN,NaN,NaN,Brentford,NaN,Elland Road,NaN,Head-to-Head,NaN,Watch
207,21.0,Sun,2023-01-22,16:30,Arsenal,NaN,NaN,NaN,Manchester Utd,NaN,Emirates Stadium,NaN,Head-to-Head,NaN,Watch
240,25.0,Fri,2023-02-24,20:00,Fulham,NaN,NaN,NaN,Wolves,NaN,Craven Cottage,NaN,Head-to-Head,NaN,Watch


#### Player Columns

In [ ]:
def scrap_player(player_id, player_name):
  player_url = f'https://fbref.com/en/players/{player_id}/{player_name}'
  # scrape the html content
  html_doc = get_html_document(player_url)
  soup = bs4.BeautifulSoup(html_doc, 'html.parser')
  return soup

In [ ]:
def get_player_table(player_id, player_name, table_category='all_stats_standard'):
  # scrape the html content
  soup = scrap_player(player_id, player_name)
  print(soup)
  # find tag with id: all_stats_standard
  html_std_stats = soup.find(id=table_category)
  # remove <tfoot> and its children
  for tfoot in soup.find_all('tfoot'):
    tfoot.decompose()
  # get the table
  table = pd.read_html(str(html_std_stats))
  return table[0]

### Storing Data

#### Necessary Functions

In [ ]:
def scrap_team(team_id, team_name):
  team_url = f'https://fbref.com/en/squads/{team_id}/{team_name}-Stats'
  html_doc = get_html_document(team_url)
  soup = bs4.BeautifulSoup(html_doc, 'html.parser')
  return soup

In [ ]:
def clean_team_stats_table(raw_table):
  # remove the multi-index
  # ex: column name ('Playing Time', 'MP') becomes ('Playing Time - MP')
  for col in raw_table.columns:
    if 'unnamed' in col[0].lower():
      continue
    new_colname = col[0] + ' - ' + col[1]

    # column reassignment is used instead of rename because rename changes 
    # all the columns with the desired name
    raw_table[('', new_colname)] = raw_table[col]
    raw_table.drop(columns=[col], inplace=True, axis=1)

  # change the column structure to make it just 1 level
  raw_table = raw_table.droplevel(level=0, axis=1)

  # drop unneeded columns & rows
  raw_table = raw_table.drop(["Matches"], axis=1)
  raw_table = raw_table[
      (raw_table["Player"] != "Squad Total") &
      (raw_table["Player"] != "Opponent Total")
  ]

  # Change NaN values to 0
  raw_table = raw_table.fillna(0)
  
  # Convert string values to numeric
  for col in raw_table.columns:
    raw_table[col] = pd.to_numeric(
        raw_table[col], 
        errors='ignore',
    )
  
  return raw_table

In [ ]:
def create_player_columns(team_id="822bd0ba", team_name="Liverpool"):
  # get html doc of the team
  team_soup = scrap_team(team_id, team_name)

  # get html table of playing time
  stats_list = [
    "standard",
    "keeper",
    "keeper_adv",
    "shooting",
    "passing",
    "passing_types",
    "gca",
    "defense",
    "possession",
    "playing_time",
    "misc",
  ]
  stats_table_list = []

  for stats in stats_list:
    # get individual table of each statistic category
    table_soup = team_soup.find('table', attrs={'id': f"stats_{stats}_9"})
    table_raw = pd.read_html(str(table_soup))[0]
    table_clean = clean_team_stats_table(table_raw)

    # rename the columns to mark from where the columns was obtained
    table_clean = table_clean.set_index("Player")
    for col in table_clean.columns:
      table_clean.rename(columns={col: f"{stats} - {col}"}, inplace=True)
    table_clean = table_clean.reset_index("Player")

    stats_table_list.append(table_clean)

  # join/merge all the tables
  df_players = stats_table_list[0]
  for stats_table in stats_table_list[1:]:
    # merge
    df_players = df_players.merge(stats_table.set_index("Player"), on="Player", how="outer",)

  # # convert into 1 row
  # df_players_one_row = df_players.set_index("Player").unstack().to_frame().T
  # # swap axis level of the columns
  # df_players_one_row = df_players_one_row.swaplevel(0, axis=1)
  # # sort the columns based on the player's name
  # df_players_one_row = df_players_one_row.sort_values("Player", axis=1)

  # # add new column name to determine the club the players belong to
  # # the club information is placed as new column level
  # df_players_one_row.columns = pd.MultiIndex.from_product([[team_name], df_players_one_row])


  #### Convert into 1 row
  # df_players_one_row = df_players.unstack().to_frame().T
  # df_players_one_row = df_players_one_row.swaplevel(1, axis=1)
  # df_players_one_row = df_players_one_row.sort_values("Player", axis=1)
  # df_players_one_row.columns.names = ["Club", "Player", "Statistic"]

  df_players_one_row = df_players.set_index("Player")
  df_players_one_row.columns = pd.MultiIndex.from_product([[team_name], df_players_one_row.columns])
  # pakai column name Club -- Player - Statistics
  df_players_one_row = df_players_one_row.unstack().to_frame().T
  df_players_one_row = df_players_one_row.swaplevel(1, axis=1).sort_values("Player", axis=1)
  df_players_one_row.columns.names = ["Club", "Player", "Statistic"]
  ###


  #### reduce the column levels further
  ## this thing makes the process from ~2 secs to ~2 mins 30 secs
  ## so i commented this because that's too long
  # for col in df_players.columns:
  #   if 'unnamed' in col[0].lower():
  #     continue
  #   new_colname = col[0] + ' - ' + col[1]

  #   # column reassignment is used instead of rename because rename changes 
  #   # df_players.rename(columns={col[1]: new_colname}, inplace=True)

  #   # all the columns with the desired name
  #   df_players[('', new_colname)] = df_players[col]
  #   df_players.drop(columns=[col], inplace=True, axis=1)

  # # change the column structure to make it just 1 level
  # df_players = df_players.droplevel(level=0, axis=1)
  ####

  return df_players, df_players_one_row

In [ ]:
%%time

test_bournemouth, test_bournemouth_one_row = create_player_columns("4ba7cbea", "Bournemouth")
test_man_utd, test_man_utd_one_row = create_player_columns("19538871", "Manchester-United")
test_liverpool, test_liverpool_one_row = create_player_columns("822bd0ba", "Liverpool")
test_chelsea, test_chelsea_one_row = create_player_columns("cff3d9bb", "Chelsea")
test_arsenal, test_arsenal_one_row = create_player_columns("18bb7c10", "Arsenal")

CPU times: user 9.36 s, sys: 124 ms, total: 9.49 s
Wall time: 14.2 s


In [ ]:
test_liverpool.shape[1] == test_bournemouth.shape[1] == test_man_utd.shape[1] == \
test_chelsea.shape[1] == test_arsenal.shape[1]

True

In [ ]:
# test_liverpool.columns.tolist()

In [ ]:
print(test_bournemouth_one_row.shape)
print(test_man_utd_one_row.shape)
print(test_liverpool_one_row.shape)
print(test_chelsea_one_row.shape)
print(test_arsenal_one_row.shape)

(1, 7968)
(1, 8466)
(1, 8466)
(1, 8715)
(1, 7470)


In [ ]:
df_nm = df_next_matches.copy()

for idx, row in df_nm.iterrows():
  home = TEAM_MAPPING[row["Home"]]
  away = TEAM_MAPPING[row["Away"]]
  date = row["Date"]

  df_home, home_players = create_player_columns(team_kv_pairs[home], home)
  df_away, away_players = create_player_columns(team_kv_pairs[away], away)

  # to prevent concat giving 2 rows instead of 1
  row_ = row.to_frame().T.reset_index().drop(columns=["index"])
  # increase the number of levels of the columns
  row_.columns = pd.MultiIndex.from_product([[""], [""], row_])
  row_.columns.names = ["Club", "Player", "Statistic"]

  df_ = pd.concat(
      [home_players, away_players], 
      axis=1,
  )
  print(df_.shape)
  df_.to_csv(f"{date}_{home}-vs-{away}.csv")

  print(f"{date}_{home}-vs-{away}.csv")
  break

(1, 13197)
2023-01-19_Manchester-City-vs-Tottenham-Hotspur.csv


In [ ]:
fc = pd.read_csv(
    "/content/2023-01-19_Manchester-City-vs-Tottenham-Hotspur.csv", 
    header=[0, 1, 2],
    index_col=0
)

In [ ]:
fcc = fc.copy()

In [ ]:
fcc.columns.to_flat_index()

Index([  ('Fulham', 'Aymeric Laporte', 'keeper - Performance - Save%'),
                           ('Fulham', 'Aymeric Laporte', 'misc - 90s'),
          ('Fulham', 'Aymeric Laporte', 'defense - Tackles - Mid 3rd'),
            ('Fulham', 'Aymeric Laporte', 'passing - Total - TotDist'),
          ('Fulham', 'Aymeric Laporte', 'keeper_adv - Sweeper - #OPA'),
          ('Fulham', 'Aymeric Laporte', 'possession - Dribbles - Mis'),
          ('Fulham', 'Aymeric Laporte', 'defense - Tackles - Att 3rd'),
        ('Fulham', 'Aymeric Laporte', 'standard - Playing Time - 90s'),
                           ('Fulham', 'Aymeric Laporte', 'misc - Age'),
         ('Fulham', 'Aymeric Laporte', 'keeper_adv - Launched - Cmp%'),
       ...
           ('Fulham', 'Yves Bissouma', 'keeper_adv - Expected - PSxG'),
             ('Fulham', 'Yves Bissouma', 'standard - Expected - npxG'),
            ('Fulham', 'Yves Bissouma', 'defense - Vs Dribbles - Tkl'),
        ('Fulham', 'Yves Bissouma', 'passing_types - 

In [ ]:
fcc.columns = ["".join(a) for a in fcc.columns.to_flat_index()]

In [ ]:
fcc.to_csv("DEFAULTFLATINDEX-2023-01-18_Crystal-Palace-vs-Manchester-United.csv")

In [ ]:
fc1 = fc.copy()
fc2 = fc.copy()

print(fc1.equals(fc2))
print(fc1.equals(pd.DataFrame()))

True
False


## Testing

### Dictionaries

In [ ]:
LEAGUE_CODE = {
    "Premier-League": 9,
    "La-Liga": 12,
    "Serie-A": 11,
    "Ligue-1": 13,
    "Bundesliga": 20,
}

TEAM_DISPLAY = {
    "Arsenal": "Arsenal",
    "Aston Villa": "Aston-Villa",
    "Bournemouth": "Bournemouth",
    "Brentford": "Brentford",
    "Brighton": "Brighton-and-Hove-Albion",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal-Palace",
    "Everton": "Everton",
    "Fulham": "Fulham",
    "Leeds United": "Leeds-United",
    "Leicester City": "Leicester-City",
    "Liverpool": "Liverpool",
    "Manchester City": "Manchester-City",
    "Manchester Utd": "Manchester-United",
    "Newcastle Utd": "Newcastle-United",
    "Nott'ham Forest": "Nottingham-Forest",
    "Southampton": "Southampton",
    "Tottenham": "Tottenham-Hotspur",
    "West Ham": "West-Ham-United",
    "Wolves": "Wolverhampton-Wanderers",
}

TEAM_CODE = {
    'Aston-Villa': '8602292d',
    'Tottenham-Hotspur': '361ca564',
    'Crystal-Palace': '47c64c55',
    'Liverpool': '822bd0ba',
    'Bournemouth': '4ba7cbea',
    'Arsenal': '18bb7c10',
    'Southampton': '33c895d4',
    'Leicester-City': 'a2d435b3',
    'West-Ham-United': '7c21e445',
    'Everton': 'd3fd31cc',
    'Brentford': 'cd051869',
    'Brighton-and-Hove-Albion': 'd07537b9',
    'Fulham': 'fd962109',
    'Wolverhampton-Wanderers': '8cec06e1',
    'Nottingham-Forest': 'e4a775cb',
    'Manchester-United': '19538871',
    'Newcastle-United': 'b2b47a98',
    'Leeds-United': '5bfb9659',
    'Chelsea': 'cff3d9bb',
    'Manchester-City': 'b8fd03ef'
}

### table_1

In [ ]:
def scrap_new_fixtures(league_fixtures_url: str) -> pd.DataFrame:
    """Get the new league fixtures information
    Parameters
    ----------
    league_url : string
        The url of the league fixtures 
    Returns
    ----------
    pd.DataFrame
        pandas DataFrame for Table 1
    """
    # scrap via beautifulsoup4
    html_doc = get_html_document(league_fixtures_url)
    soup = bs4.BeautifulSoup(html_doc, 'html.parser')
    
    # scrap directly the table via pandas
    pl_fixtures = pd.read_html(league_fixtures_url)
    df_table_1 = pd.DataFrame(pl_fixtures[0])
    df_table_1 = df_table_1.dropna(axis=0, how='all').reset_index().drop(columns=["index"])
    df_table_1 = df_table_1[df_table_1["Notes"] != "Match Postponed"]
    df_table_1["Match Report Link"] = 'empty'
    
    past_matches = []
    temp_soup = soup.find_all('table')[0]

    for idx, row in df_table_1[df_table_1["Match Report"] == "Match Report"].iterrows():
        home = row["Home"]
        home = TEAM_DISPLAY[home]
        away = row["Away"]
        away = TEAM_DISPLAY[away]

        past_matches.append((home, away))

        for link in temp_soup.find_all('a', attrs={'href': re.compile("^/en/matches/")}):
            # ex: "/en/matches/073227b6/Merseyside-Derby-Everton-Liverpool-September-3-2022-Premier-League"
            last_link = link.get('href').split('/')[-1]

            # ex: "Merseyside-Derby-Everton-Liverpool-September-3-2022-Premier-League"
            if "Premier-League" not in last_link:
                continue

            last_link = last_link.split("-Derby-")[-1]
            # ex: "Everton-Liverpool-September-3-2022-Premier-League"
            
            if last_link.startswith(f"{home}-{away}"):
                df_table_1.loc[idx, "Match Report Link"] = link.get('href')
                break
    
    return df_table_1

In [ ]:
league_fixtures_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
table_1 = scrap_new_fixtures(league_fixtures_url=league_fixtures_url)

In [ ]:
table_1

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN,/en/matches/6713c1dc/Fulham-Liverpool-August-6...
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN,/en/matches/09d8a999/Tottenham-Hotspur-Southam...
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,Match Report,NaN,/en/matches/1ac96eb4/Newcastle-United-Nottingh...
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN,/en/matches/82702941/Leeds-United-Wolverhampto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN,empty
376,38.0,Sun,2023-05-28,15:00,Leicester City,NaN,NaN,NaN,West Ham,NaN,King Power Stadium,NaN,Head-to-Head,NaN,empty
377,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN,empty
378,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN,empty


### Get Just-Finished Matches

In [ ]:
table_1_test = pd.read_csv("table_1_test.csv", index_col=0)
table_2_test = pd.read_csv("table_2_test.csv", index_col=0)

In [ ]:
cond = table_1_test["Match Report Link"] != table_2_test["Match Report Link"]

In [ ]:
df_new_finished_matches = table_1_test.loc[cond, :]

In [ ]:
df_new_finished_matches

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link,League
195,20.0,Sun,2023-01-15,14:00,Newcastle Utd,2.2,1–0,1.1,Fulham,52247.0,St James' Park,Robert Jones,Match Report,NaN,/en/matches/f93d11e7/Newcastle-United-Fulham-J...,Premier-League
196,20.0,Sun,2023-01-15,16:30,Tottenham,1.6,0–2,1.8,Arsenal,61870.0,Tottenham Hotspur Stadium,Craig Pawson,Match Report,NaN,/en/matches/4fc8b7b2/North-London-Derby-Totten...,Premier-League


### Get Next Matches

In [ ]:
table_2 = table_1.copy()

df_next_matches = table_2[table_2["Match Report"] != "Match Report"]

In [ ]:
df_next_matches.head(5)

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
198,7.0,Thu,2023-01-19,20:00,Manchester City,NaN,NaN,NaN,Tottenham,NaN,Etihad Stadium,NaN,Head-to-Head,NaN,empty
199,21.0,Sat,2023-01-21,12:30,Liverpool,NaN,NaN,NaN,Chelsea,NaN,Anfield,NaN,Head-to-Head,NaN,empty
200,21.0,Sat,2023-01-21,15:00,Bournemouth,NaN,NaN,NaN,Nott'ham Forest,NaN,Vitality Stadium,NaN,Head-to-Head,NaN,empty
201,21.0,Sat,2023-01-21,15:00,Southampton,NaN,NaN,NaN,Aston Villa,NaN,St. Mary's Stadium,NaN,Head-to-Head,NaN,empty
202,21.0,Sat,2023-01-21,15:00,Leicester City,NaN,NaN,NaN,Brighton,NaN,King Power Stadium,NaN,Head-to-Head,NaN,empty


In [ ]:
## next few matches

clubs_recorded = []
df_ = []

for idx, row in df_next_matches.iterrows():
  club1 = row["Home"]
  club2 = row["Away"]
  
  if club1 in clubs_recorded or club2 in clubs_recorded:
    break

  df_.append(row)
  clubs_recorded.append(club1)
  clubs_recorded.append(club2)

df_next_matches = pd.DataFrame(df_)

In [ ]:
df_next_matches

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link
198,7.0,Thu,2023-01-19,20:00,Manchester City,NaN,NaN,NaN,Tottenham,NaN,Etihad Stadium,NaN,Head-to-Head,NaN,empty
199,21.0,Sat,2023-01-21,12:30,Liverpool,NaN,NaN,NaN,Chelsea,NaN,Anfield,NaN,Head-to-Head,NaN,empty
200,21.0,Sat,2023-01-21,15:00,Bournemouth,NaN,NaN,NaN,Nott'ham Forest,NaN,Vitality Stadium,NaN,Head-to-Head,NaN,empty
201,21.0,Sat,2023-01-21,15:00,Southampton,NaN,NaN,NaN,Aston Villa,NaN,St. Mary's Stadium,NaN,Head-to-Head,NaN,empty
202,21.0,Sat,2023-01-21,15:00,Leicester City,NaN,NaN,NaN,Brighton,NaN,King Power Stadium,NaN,Head-to-Head,NaN,empty
203,21.0,Sat,2023-01-21,15:00,West Ham,NaN,NaN,NaN,Everton,NaN,London Stadium,NaN,Head-to-Head,NaN,empty
204,21.0,Sat,2023-01-21,17:30,Crystal Palace,NaN,NaN,NaN,Newcastle Utd,NaN,Selhurst Park,NaN,Head-to-Head,NaN,empty


### Generate Pre-Match Infos (CSV)

In [ ]:
def scrap_team(team_id, team_name):
  team_url = f'https://fbref.com/en/squads/{team_id}/{team_name}-Stats'
  html_doc = get_html_document(team_url)
  soup = bs4.BeautifulSoup(html_doc, 'html.parser')
  return soup

In [ ]:
def clean_team_stats_table(raw_table):
  # remove the multi-index
  # ex: column name ('Playing Time', 'MP') becomes ('Playing Time - MP')
  for col in raw_table.columns:
    if 'unnamed' in col[0].lower():
      continue
    new_colname = col[0] + ' - ' + col[1]

    # column reassignment is used instead of rename because rename changes 
    # all the columns with the desired name
    raw_table[('', new_colname)] = raw_table[col]
    raw_table.drop(columns=[col], inplace=True, axis=1)

  # change the column structure to make it just 1 level
  raw_table = raw_table.droplevel(level=0, axis=1)

  # drop unneeded columns & rows
  raw_table = raw_table.drop(["Matches"], axis=1)
  raw_table = raw_table[
      (raw_table["Player"] != "Squad Total") &
      (raw_table["Player"] != "Opponent Total")
  ]

  # Change NaN values to 0
  raw_table = raw_table.fillna(0)
  
  # Convert string values to numeric
  for col in raw_table.columns:
    raw_table[col] = pd.to_numeric(
        raw_table[col], 
        errors='ignore',
    )
  
  return raw_table

In [ ]:
def create_player_columns(team_id="822bd0ba", team_name="Liverpool"):
  # get html doc of the team
  team_soup = scrap_team(team_id, team_name)

  # get html table of playing time
  stats_list = [
    "standard",
    "keeper",
    "keeper_adv",
    "shooting",
    "passing",
    "passing_types",
    "gca",
    "defense",
    "possession",
    "playing_time",
    "misc",
  ]
  stats_table_list = []

  for stats in stats_list:
    # get individual table of each statistic category
    table_soup = team_soup.find('table', attrs={'id': f"stats_{stats}_9"})
    table_raw = pd.read_html(str(table_soup))[0]
    table_clean = clean_team_stats_table(table_raw)

    # rename the columns to mark from where the columns was obtained
    table_clean = table_clean.set_index("Player")
    for col in table_clean.columns:
      table_clean.rename(columns={col: f"{stats} - {col}"}, inplace=True)
    table_clean = table_clean.reset_index("Player")

    stats_table_list.append(table_clean)

  # join/merge all the tables
  df_players = stats_table_list[0]
  for stats_table in stats_table_list[1:]:
    # merge
    df_players = df_players.merge(stats_table.set_index("Player"), on="Player", how="outer",)

  # # convert into 1 row
  # df_players_one_row = df_players.set_index("Player").unstack().to_frame().T
  # # swap axis level of the columns
  # df_players_one_row = df_players_one_row.swaplevel(0, axis=1)
  # # sort the columns based on the player's name
  # df_players_one_row = df_players_one_row.sort_values("Player", axis=1)

  # # add new column name to determine the club the players belong to
  # # the club information is placed as new column level
  # df_players_one_row.columns = pd.MultiIndex.from_product([[team_name], df_players_one_row])


  #### Convert into 1 row
  # df_players_one_row = df_players.unstack().to_frame().T
  # df_players_one_row = df_players_one_row.swaplevel(1, axis=1)
  # df_players_one_row = df_players_one_row.sort_values("Player", axis=1)
  # df_players_one_row.columns.names = ["Club", "Player", "Statistic"]

  df_players_one_row = df_players.set_index("Player")
  df_players_one_row.columns = pd.MultiIndex.from_product([[team_name], df_players_one_row.columns])
  # pakai column name Club -- Player - Statistics
  df_players_one_row = df_players_one_row.unstack().to_frame().T
  df_players_one_row = df_players_one_row.swaplevel(1, axis=1).sort_values("Player", axis=1)
  df_players_one_row.columns.names = ["Club", "Player", "Statistic"]
  ###


  #### reduce the column levels further
  ## this thing makes the process from ~2 secs to ~2 mins 30 secs
  ## so i commented this because that's too long
  # for col in df_players.columns:
  #   if 'unnamed' in col[0].lower():
  #     continue
  #   new_colname = col[0] + ' - ' + col[1]

  #   # column reassignment is used instead of rename because rename changes 
  #   # df_players.rename(columns={col[1]: new_colname}, inplace=True)

  #   # all the columns with the desired name
  #   df_players[('', new_colname)] = df_players[col]
  #   df_players.drop(columns=[col], inplace=True, axis=1)

  # # change the column structure to make it just 1 level
  # df_players = df_players.droplevel(level=0, axis=1)
  ####

  return df_players, df_players_one_row

In [ ]:
!mkdir "raw/"
!mkdir "raw/Premier League"
!mkdir "raw/Premier League/Matches"

In [ ]:
SAVE_DIR = "/content/raw/Premier League/Matches"

def get_pre_match_info(df_next_matches, save_dir):
  df_nm = df_next_matches.copy()

  for idx, row in df_nm.head(2).iterrows():
    home = TEAM_MAPPING[row["Home"]]
    away = TEAM_MAPPING[row["Away"]]
    date = row["Date"]

    df_home, home_players = create_player_columns(team_kv_pairs[home], home)
    df_away, away_players = create_player_columns(team_kv_pairs[away], away)

    # to prevent concat giving 2 rows instead of 1
    row_ = row.to_frame().T.reset_index().drop(columns=["index"])
    # increase the number of levels of the columns
    row_.columns = pd.MultiIndex.from_product([[""], [""], row_])
    row_.columns.names = ["Club", "Player", "Statistic"]

    df_ = pd.concat(
        [home_players, away_players], 
        axis=1,
    )
    print(df_.shape)
    df_.to_csv(f"{save_dir}/{date}_{home}-vs-{away}.csv")

    print(f"Saved to: {save_dir}/{date}_{home}-vs-{away}.csv")

In [ ]:
get_pre_match_info(df_next_matches, save_dir=SAVE_DIR)

(1, 13197)
Saved to: /content/raw/Premier League/Matches/2023-01-19_Manchester-City-vs-Tottenham-Hotspur.csv
(1, 17181)
Saved to: /content/raw/Premier League/Matches/2023-01-21_Liverpool-vs-Chelsea.csv


### Generate Post-Match Infos (JSON)

In [ ]:
df_new_finished_matches

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,Match Report Link,League
195,20.0,Sun,2023-01-15,14:00,Newcastle Utd,2.2,1–0,1.1,Fulham,52247.0,St James' Park,Robert Jones,Match Report,NaN,/en/matches/f93d11e7/Newcastle-United-Fulham-J...,Premier-League
196,20.0,Sun,2023-01-15,16:30,Tottenham,1.6,0–2,1.8,Arsenal,61870.0,Tottenham Hotspur Stadium,Craig Pawson,Match Report,NaN,/en/matches/4fc8b7b2/North-London-Derby-Totten...,Premier-League


In [ ]:
def get_match_data_as_dict(
    match_link="/en/matches/de515487/Liverpool-Bournemouth-August-27-2022-Premier-League",
    home="Liverpool",
    away="Bournemouth"
):
  df_ = pd.read_html("https://fbref.com" + match_link)
  desired_column = ["Player", "Pos", "Age", "Min"]
  post_match_dict = {}
  post_match_dict[home] = {}
  post_match_dict[away] = {}

  home_players = df_[3].copy()
  home_players.columns = [col[1] for col in home_players.columns.to_flat_index()]
  for idx, row in home_players[desired_column].iloc[:-1].iterrows():
    post_match_dict[home][row["Player"]] = {}
    post_match_dict[home][row["Player"]]["Pos"] = row["Pos"]
    post_match_dict[home][row["Player"]]["Age"] = row["Age"]
    post_match_dict[home][row["Player"]]["Min"] = row["Min"]

  away_players = df_[10].copy()
  away_players.columns = [col[1] for col in away_players.columns.to_flat_index()]
  
  for idx, row in away_players[desired_column].iloc[:-1].iterrows():
    post_match_dict[away][row["Player"]] = {}
    post_match_dict[away][row["Player"]]["Pos"] = row["Pos"]
    post_match_dict[away][row["Player"]]["Age"] = row["Age"]
    post_match_dict[away][row["Player"]]["Min"] = row["Min"]

  return post_match_dict

In [ ]:
## Obtain lists of Match Result Links

for idx, row in df_new_finished_matches.iterrows():
  match_report_link = row["Match Report Link"]
  home = TEAM_MAPPING[row["Home"]]
  away = TEAM_MAPPING[row["Away"]]
  date = row["Date"]

  post_match_dict = get_match_data_as_dict(match_report_link, home, away)

  with open(f"{SAVE_DIR}/{date}_{home}-vs-{away}.json", 'w') as fp:
    json.dump(post_match_dict, fp, indent=4)
  print(f"Saved to: {SAVE_DIR}/{date}_{home}-vs-{away}.json")

Saved to: /content/raw/Premier League/Matches/2023-01-15_Newcastle-United-vs-Fulham.json
Saved to: /content/raw/Premier League/Matches/2023-01-15_Tottenham-Hotspur-vs-Arsenal.json


In [ ]:
pd.DataFrame().to_csv("raw/Premier League/Matches/test.csv")

In [ ]:
if False:
  print(2)
else:
  print(3)

3


## Other Leagues

In [ ]:
epl = (9, 'Premier-League')
la_liga = (12, 'La-Liga')
serie_a = (11, 'Serie-A')
ligue_1 = (13, 'Ligue-1')
bundesliga = (20, 'Bundesliga')

In [ ]:
test_cadiz = {"Cádiz": "cek"}

In [ ]:
test_cadiz["Cádiz"]

'cek'

In [ ]:
df1 = pd.DataFrame({
    "c1": [1, 2, 3],
    "c2": [1, 2, 4]
})

df2 = pd.DataFrame({
    "c1": [1, 2, 3],
    "c2": [1, 2, 5]
})

In [ ]:
(df1["c2"] == df2["c2"]).any()

True